# Arbeitspaket (AP) 3: Management & Nutzung Räumliche Daten

### Angaben Studierende(r) (fehlende Angaben ergänzen)

<table>
  <tr>
    <td>Vorname:</td>
    <td>Dante</td>
  </tr>
  <tr>
    <td>Nachname:</td>
    <td>Stoppini</td>
  </tr>
  <tr>
    <td>Immatrikulationsnummer:</td>
    <td>17-653-874</td>
  </tr>
  <tr>
    <td>Modul:</td>
    <td>Data Science</td>
  </tr>
  <tr>
    <td>Prüfungsdatum / Raum / Zeit:</td>
    <td>06.10.2025 / Raum: MU O2.001 / 8:00 – 11:45</td>
  </tr>
  <tr>
    <td>Erlaubte Hilfsmittel:</td>
    <td>w.MA.XX.DS.25HS (Data Science)<br>Open Book, Eigener Computer, Internet-Zugang</td>
  </tr>
  <tr>
  <td>Nicht erlaubt:</td>
  <td>Nicht erlaubt ist der Einsatz beliebiger Formen von generativer KI (z.B. Copilot, ChatGPT) <br> sowie beliebige Formen von Kommunikation oder Kollaboration mit anderen Menschen.</td>
</tr>
</table>

## Bewertungskriterien

### <b style="color: gray;">(max. erreichbare Punkte: 40)</b>

<table>
  <thead>
    <tr>
      <th>Kategorie</th>
      <th>Beschreibung</th>
      <th>Punkteverteilung</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>Code nicht lauffähig oder Ergebnisse nicht sinnvoll</td>
      <td>Der Code enthält Fehler, die verhindern, dass er ausgeführt werden kann (z.B. Syntaxfehler) oder es werden Ergebnisse ausgegeben, welche nicht zur Fragestellung passen.</td>
      <td>0 Punkte</td>
    </tr>
    <tr>
      <td>Code lauffähig, aber mit gravierenden Mängeln</td>
      <td>Der Code läuft, aber die Ergebnisse sind aufgrund wesentlicher Fehler unvollständig (z.B. fehlende Joins, gravierende Fehler in SQL-Abfragen). Nur geringer Fortschritt erkennbar.</td>
      <td>25% der max. erreichbaren Punkte</td>
    </tr>
    <tr>
      <td>Code lauffähig, aber mit mittleren Mängeln</td>
      <td>Der Code läuft und liefert teilweise korrekte Ergebnisse, aber es gibt grössere Fehler (z.B. fehlende Spalten, unvollständige SQL-Abfragen). Die Ergebnisse sind nachvollziehbar, aber unvollständig oder ungenau.</td>
      <td>50% der max. erreichbaren Punkte</td>
    </tr>
    <tr>
      <td>Code lauffähig, aber mit minimalen Mängeln</td>
      <td>Der Code läuft und liefert ein weitgehend korrektes Ergebnis, aber kleinere Fehler (z.B. falsche oder fehlende Sortierung, Rundung von Werten falsch) beeinträchtigen die Vollständigkeit des Ergebnisses.</td>
      <td>75% der max. erreichbaren Punkte</td>
    </tr>
    <tr>
      <td>Code lauffähig und korrekt</td>
      <td>Der Code läuft einwandfrei und liefert das korrekte Ergebnis ohne Mängel.</td>
      <td>100% der max. erreichbaren Punkte</td>
    </tr>
  </tbody>
</table>


#### <b>Python Libraries und Settings</b>

In [1]:
# Libraries
import os
import folium
import pandas as pd
import geopandas as gpd
from IPython.display import display, IFrame
from sqlalchemy import create_engine, text

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

print(os.getcwd())


/workspaces/python_postgresql_postgis/AP03


## <b>Vorbereitung (Hinweis: dieser Teil wird <u>nicht</u> bewertet)</b>

#### <b>1.) Starten Sie eine GitHub Codespaces Instanz auf Basis Ihres Forks des folgenden GitHub Repositories:</b>

##### GitHub-Repository: https://github.com/mario-gellrich-zhaw/python_postgresql_postgis

##### <span style="color: red;"><b>WICHTIG!!! Verwenden Sie eine GitHub Codespaces Instanz mit ausreichend Arbeitsspeicher (4core, 16GB RAM).</b></span>

<b>Hinweis:</b> 
- Im Unterricht wurden bereits sämtliche Installationen und Einstellungen inkl. der Registrierung des Datenbank Servers auf pgAdmin vorgenommen.
- Falls Sie die Codespaces-Instanz neu erstellen müssen, folgen Sie bitte den detaillierten Erklärungen auf der README-Seite des GitHub Repositories.

#### <b>2.) Erstellen und Testen Sie die Datenbankverbindung mit der 'osm_switzerland' Datenbank.</b>

In [2]:
# Set up Database Connection
user = "pgadmin"
password = "geheim"
host = "localhost"
port = "5432"
database = "osm_switzerland"

# Erstellen der Connection URL
db_connection_url = "postgresql://" + user + ":" + password +\
                    "@" + host + ":" + port + "/" + database

# Erstellen SQLAlchemy Engine
engine = create_engine(db_connection_url)

# Test der Connection
try:
    with engine.connect() as connection:
        result = connection.execute(text("SELECT current_database();"))
        db_name = result.scalar()
        print("Aktuelle Datenbank:", db_name)
except Exception as e:
    print("Fehler bei der Verbindung:", e)

# Verbindung trennen
engine.dispose()

Aktuelle Datenbank: osm_switzerland


#### <b>3.) Legen Sie einen neuen Order 'maps' im Arbeitsverzeichnis an. In diesem werden Ihre generierten Karten abgespeichert.</b>

<b style="color:red;">Hinweis: Der Order 'maps' inklusive die erzeugten Karten muss als Teil des AP03 mit auf EXAM-Moodle abgegeben werden!</b>

In [3]:
# Anlegen eines Ordners 'maps'
if not os.path.exists('maps'):
    os.makedirs('maps')
    print("Ordner 'maps' wurde erstellt.")
else:
    print("Ordner 'maps' existiert bereits und wird nicht neu angelegt!")

Ordner 'maps' wurde erstellt.


## <b>Aufgaben (Dieser Teil wird bewertet!)</b>
<b>Hinweise zu den folgenden Aufgabenstellungen:</b>
<ul>
  <li>In diesem Jupyter Notebook gibt es jeweils zwei Code-Zellen pro Aufgabe:</li>
  <ol>
    <li>Eine Codezelle mit Python-Code für die SQL-Abfrage.</li>
    <li>Eine Codezelle mit Python-Code für die Kartendarstellung der Ergebnisse der jeweiligen SQL-Abfrage.</li>
  </ol>
  <li>In den Codezellen für die SQL-Abfrage muss jeweils die SQL-Statement ergänzt werden.</li>
  <li>In den Codezellen für die Kartendarstellung muss nur dann der Python Code ergänzt werden, wenn in der Aufgabe danach gefragt wird.</li>
</ul>
<b style="color:red;">Beachten Sie, dass für die Punktevergabe auch die weiteren Anforderungen zu den Fragen unter 'Details zur Aufgabenstellung' herangezogen werden.</b>

#### <b>Aufgabe (1): Erstellen Sie eine Abfrage der Spitäler.</b>
<b>Details zur Aufgabenstellung:</b>
- Sie finden die benötigten Daten in der Tabelle 'planet_osm_polygon'.
- Stellen sie in der Ergebnistabelle die Spalten: osm_id, amenity sowie die transformierte Geometrie als Spalte geom dar.
- Tipps:
    - Die Geometry wird mit Hilfe der Funktion st_transform() transformiert, z.B.: *st_transform(p.way, 4326) AS geom*.
    - Die Daten sind mit dem key:value Paar amenity='hospital' in der OpenStreetMap Map-Feature Übersicht angegeben.
    - vgl: https://wiki.openstreetmap.org/wiki/Map_features

<b style="color: gray;">(max. erreichbare Punkte: 4)</b>

In [4]:
# Engine für Datenbankverbindung erstellen
engine = create_engine(db_connection_url)

# Ergänzen Sie die SQL-Abfrage, um die Aufgabe zu lösen
sql = """SELECT
                p.osm_id,
                p.amenity,
                st_transform(p.way, 4326) AS geom
        FROM
                public.planet_osm_polygon AS p
        WHERE
                p.amenity='hospital'"""

# Ergebnis in GeoDataFrame abspeichern
gdf_hospital = gpd.GeoDataFrame.from_postgis(sql, engine)

# Datenbankverbindung trennen
engine.dispose()

# Zeigen des GeoDataFrames
gdf_hospital.head(10)

,osm_id,amenity,geom
0,165847373,hospital,"POLYGON ((8.72864 47.51029, 8.72927 47.51012, ..."
1,612288154,hospital,"POLYGON ((8.54828 47.38012, 8.5483 47.38012, 8..."
2,125667230,hospital,"POLYGON ((8.57592 47.3519, 8.57597 47.35179, 8..."
3,28530564,hospital,"POLYGON ((8.79935 47.32184, 8.79939 47.32181, ..."
4,317763534,hospital,"POLYGON ((8.58826 47.26274, 8.58826 47.26272, ..."
5,-9065183,hospital,"POLYGON ((8.45701 47.28345, 8.4573 47.28318, 8..."
6,-9065183,hospital,"POLYGON ((8.45507 47.28548, 8.45516 47.28523, ..."
7,651732956,hospital,"POLYGON ((8.46535 47.27759, 8.46552 47.27739, ..."
8,609537142,hospital,"POLYGON ((8.51821 47.39843, 8.51822 47.39839, ..."
9,42156983,hospital,"POLYGON ((8.5172 47.41482, 8.51749 47.41474, 8..."


##### <b>Kartendarstellung Ergebnis (nur anpassen, falls in der Aufgabe danach gefragt wird)</b>

In [5]:
# Projektion definieren (WGS84)
if gdf_hospital.crs is None:
    gdf_hospital.set_crs(epsg=4326, inplace=True)
else:
    pass

# Latitude und Longitude für die Zentrierung der Karte ermitteln
centroids = gdf_hospital.geometry.centroid
lon = centroids.x.mean()
lat = centroids.y.mean()

# Initialisieren der Map
m = folium.Map(location=[lat, lon], 
               zoom_start=11, 
               tiles='CartoDBPositron')

# Map settings
folium.GeoJson(
    gdf_hospital,
    name='map'
).add_to(m)

folium.LayerControl().add_to(m)

# Save the map to an HTML file
m.save('maps/map_hospital.html')
print("Die Karte wurde unter 'maps/map_hospital.html' gespeichert.")

Die Karte wurde unter 'maps/map_hospital.html' gespeichert.


#### <b>Aufgabe (2) Erstellen Sie eine Abfrage der SBB Bahnhöfe.</b>
<b>Details zur Aufgabenstellung:</b>
- Verwenden Sie die Tabelle 'planet_osm_point'.
- Schränken Sie nach Bahnhöfen der SBB ein.
- Stellen Sie in der Ergebnistabelle die Spalten: osm_id, name, operator, public_transport und die transformierte Geometrie als Spalte geom dar.
- Tipp: Die Daten sind mit dem key:value Paar public_transport='station' in der OpenStreetMap Map-Feature Übersicht angegeben.
-  vgl: https://wiki.openstreetmap.org/wiki/Map_features

<b style="color: gray;">(max. erreichbare Punkte: 4)</b>

In [9]:
# Engine für Datenbankverbindung erstellen


# Ergänzen Sie die SQL-Abfrage, um die Aufgabe zu lösen
sql = """SELECT
                p.osm_id,
                p.name,
                p.operator,
                p.public_transport,
                st_transform(p.way, 4326) AS geom
        FROM
                public.planet_osm_point AS p
        WHERE p.operator = 'SBB' AND
                p.public_transport='station'"""

# Ergebnis in GeoDataFrame abspeichern
gdf_stations = gpd.GeoDataFrame.from_postgis(sql, engine)

# Datenbankverbindung trennen


# Zeigen des GeoDataFrames
gdf_stations.head(10)

,osm_id,name,operator,public_transport,geom
0,4574932116,Rüti,SBB,station,POINT (8.85436 47.26035)
1,2414238691,Tann-Dürnten,SBB,station,POINT (8.85245 47.26845)
2,2141146352,Wald ZH,SBB,station,POINT (8.91396 47.27273)
3,2443333883,Steg,SBB,station,POINT (8.93238 47.35388)
4,2141115746,Fischenthal,SBB,station,POINT (8.92329 47.33342)
5,762194590,Bauma,SBB,station,POINT (8.87821 47.36882)
6,2443368760,Saland,SBB,station,POINT (8.85423 47.39426)
7,2446055623,Wila,SBB,station,POINT (8.84854 47.41778)
8,2446048209,Turbenthal,SBB,station,POINT (8.84393 47.43731)
9,1352757256,Wiesendangen,SBB,station,POINT (8.77602 47.52553)


##### <b>Kartendarstellung Ergebnis (nur anpassen, falls in der Aufgabe danach gefragt wird)</b>

In [10]:
# Projektion definieren (WGS84)
if gdf_stations.crs is None:
    gdf_stations.set_crs(epsg=4326, inplace=True)
else:
    pass

# Latitude und Longitude für die Zentrierung der Karte ermitteln
centroids = gdf_stations.geometry.centroid
lon = centroids.x.mean()
lat = centroids.y.mean()

# Initialisieren der Map
m = folium.Map(location=[lat, lon], 
               zoom_start=10, 
               tiles='CartoDBPositron')

# Map settings with popup
folium.GeoJson(
    gdf_stations,
    name='map'
).add_to(m)

folium.LayerControl().add_to(m)

# Save the map to an HTML file
m.save('maps/map_stations.html')
print("Die Karte wurde unter 'maps/map_stations.html' gespeichert.")

Die Karte wurde unter 'maps/map_stations.html' gespeichert.


#### <b>Aufgabe (3): Erstellen Sie eine Abfrage aller Strassen, die als 'primary', 'secondary' oder 'tertiary' klassifiziert sind mit Infos zur Art des Fahrradwegs.</b>
<b>Details zur Aufgabenstellung:</b>
- Sie finden die benötigten Informationen in der Tabelle 'planet_osm_line'.
- Stellen Sie in der Ergebnistabelle die Spalten: osm_id, highway, bicycle und die transformierte Geometrie als Spalte geom dar.
- Schränken Sie die Abfrage auf alle Strassen ein, bei denen Information zur Art des Fahrradweges vorliegen.
- Tipp: Sie finden die Informationen zur Art der Fahrradwegs in der Spalte 'bicycle'.
-  vgl: https://wiki.openstreetmap.org/wiki/Map_features

<b style="color: gray;">(max. erreichbare Punkte: 4)</b>

In [14]:
# Engine für Datenbankverbindung erstellen


# Ergänzen Sie die SQL-Abfrage, um die Aufgabe zu lösen
sql = """SELECT
                p.osm_id, p.highway, p.bicycle,
                st_transform(p.way, 4326) AS geom
        FROM
                public.planet_osm_line AS p
        WHERE 
        p.highway IN('primary', 'secondary', 'tertiary')
        AND
        p.bicycle IS NOT NULL"""

# Ergebnis in GeoDataFrame abspeichern
gdf_streets = gpd.GeoDataFrame.from_postgis(sql, engine)

# Datenbankverbindung trennen


# Zeigen des GeoDataFrames
gdf_streets.head(10)

,osm_id,highway,bicycle,geom
0,315007668,primary,use_sidepath,"LINESTRING (8.83126 47.24526, 8.83146 47.24537..."
1,1188796458,primary,use_sidepath,"LINESTRING (8.83238 47.24567, 8.83257 47.24569..."
2,1188796459,primary,use_sidepath,"LINESTRING (8.83489 47.24705, 8.83486 47.24703)"
3,315007660,primary,use_sidepath,"LINESTRING (8.83486 47.24693, 8.83496 47.247)"
4,31726656,primary,use_sidepath,"LINESTRING (8.83496 47.247, 8.83522 47.24717, ..."
5,1188796454,primary,use_sidepath,"LINESTRING (8.83529 47.24731, 8.83507 47.24717..."
6,265787852,primary,use_sidepath,"LINESTRING (8.83433 47.24651, 8.83447 47.24663..."
7,1188796450,primary,use_sidepath,"LINESTRING (8.83486 47.24703, 8.83464 47.24688..."
8,315007661,primary,use_sidepath,"LINESTRING (8.83361 47.24607, 8.83388 47.24621..."
9,315007654,primary,use_sidepath,"LINESTRING (8.83904 47.24789, 8.83933 47.24794..."


##### <b>Kartendarstellung Ergebnis (nur anpassen, falls in der Aufgabe danach gefragt wird)</b>

In [15]:
# Projektion definieren (WGS84)
if gdf_streets.crs is None:
    gdf_streets.set_crs(epsg=4326, inplace=True)
else:
    pass

# Latitude und Longitude für die Zentrierung der Karte ermitteln
centroids = gdf_streets.geometry.centroid
lon = centroids.x.mean()
lat = centroids.y.mean()

# Initialisieren der Map
m = folium.Map(location=[lat, lon], 
               zoom_start=11, 
               tiles='CartoDBPositron')

# Map settings
folium.Choropleth(
    geo_data=gdf_streets,
    name='map',
    line_weight=3,
    line_color='darkred'
).add_to(m)

folium.LayerControl().add_to(m)

# Save the map to an HTML file
m.save('maps/map_streets.html')
print("Die Karte wurde unter 'maps/map_streets.html' gespeichert.")

Die Karte wurde unter 'maps/map_streets.html' gespeichert.


#### <b>Aufgabe (4): Erstellen Sie eine Abfrage aller Gebäude in der Bahnhofstrasse in Zürich, bei denen mindestens die Postleitzahl (PLZ) vorhanden ist.</b>
<b>Details zur Aufgabenstellung:</b>
- Verwenden Sie die Tabelle 'planet_osm_polygon'.
- Stellen Sie in der Ergebnistabelle Strassenname, Haunummer, PLZ, Gemeindename sowie die transformierte Geometrie als Spalte geom dar.
- Dabei ist nicht wichtig, dass alle Adress-Attribute für jedes Gebäude vorhanden sind, nur die PLZ muss immer vorhanden sein.
- Verwenden Sie für die Darstellung als Hintergrundkarte ein Satellitenbild (ESRIWorldImagery) als maptile.
- Tipp: Gebäude sind in der Spalte 'building' klassifiziert. Mit *WHERE building IS NOT NULL* können Sie Gebäude filtern.

<b style="color: gray;">(max. erreichbare Punkte: 4)</b>

In [20]:
# Engine für Datenbankverbindung erstellen


# Ergänzen Sie die SQL-Abfrage, um die Aufgabe zu lösen
sql = """SELECT
                p.osm_id,
                p."addr:street",
                p."addr:housenumber",
                p."addr:city",
                p."addr:postcode",
                st_transform(p.way, 4326) AS geom
        FROM
                public.planet_osm_polygon AS p
        WHERE 
        p."addr:street" = 'Bahnhofstrasse'
        and p."addr:city" = 'Zürich' and
        p.building is not null and
        p."addr:postcode" is not null
       """

# Ergebnis in GeoDataFrame abspeichern
gdf_buildings = gpd.GeoDataFrame.from_postgis(sql, engine)

# Datenbankverbindung trennen


# Zeigen des GeoDataFrames
gdf_buildings.head(10)

,osm_id,addr:street,addr:housenumber,addr:city,addr:postcode,geom
0,291957917,Bahnhofstrasse,108,Zürich,8001,"POLYGON ((8.53988 47.37663, 8.54007 47.37658, ..."
1,291957916,Bahnhofstrasse,106,Zürich,8001,"POLYGON ((8.53983 47.37652, 8.53997 47.37648, ..."
2,291657083,Bahnhofstrasse,92,Zürich,8001,"POLYGON ((8.53944 47.37577, 8.53957 47.37574, ..."
3,291221102,Bahnhofstrasse,84,Zürich,8001,"POLYGON ((8.53921 47.37532, 8.53937 47.37528, ..."
4,290914347,Bahnhofstrasse,78,Zürich,8001,"POLYGON ((8.53898 47.37488, 8.53917 47.37483, ..."
5,290914348,Bahnhofstrasse,80,Zürich,8001,"POLYGON ((8.53903 47.37497, 8.53922 47.37492, ..."
6,290914345,Bahnhofstrasse,76,Zürich,8001,"POLYGON ((8.53893 47.37477, 8.53913 47.37473, ..."
7,291657084,Bahnhofstrasse,94,Zürich,8001,"POLYGON ((8.53952 47.37592, 8.53965 47.37589, ..."
8,291657085,Bahnhofstrasse,98,Zürich,8001,"POLYGON ((8.53959 47.37605, 8.53978 47.37601, ..."
9,291657079,Bahnhofstrasse,100,Zürich,8001,"POLYGON ((8.53964 47.37614, 8.53975 47.37612, ..."


##### <b>Kartendarstellung Ergebnis (nur anpassen, falls in der Aufgabe danach gefragt wird)</b>

In [21]:
# Projektion definieren (WGS84)
if gdf_buildings.crs is None:
    gdf_buildings.set_crs(epsg=4326, inplace=True)
else:
    pass

# Latitude und Longitude für die Zentrierung der Karte ermitteln
centroids = gdf_buildings.geometry.centroid
lon = centroids.x.mean()
lat = centroids.y.mean()

# Initialisieren der Map
m = folium.Map(location=[lat, lon], 
               zoom_start=16, 
               tiles='ESRIWorldImagery')

# Map settings
folium.Choropleth(
    geo_data=gdf_buildings,
    name='map',
    fill_color='orange'
).add_to(m)

folium.LayerControl().add_to(m)

# Save the map to an HTML file
m.save('maps/map_buildings.html')
print("Die Karte wurde unter 'maps/map_buildings.html' gespeichert.")

Die Karte wurde unter 'maps/map_buildings.html' gespeichert.


#### <b>Aufgabe (5): Erstellen Sie eine Abfrage des Greifensees, und generieren Sie einen Buffer um diesen See mit einer Breite von 500m.</b>
<b>Details zur Aufgabenstellung:</b>
- Sie finden die benötigten Informationen in der Tabelle 'planet_osm_polygon'.
- Stellen Sie in der Ergebnistabelle die Spalten: osm_id, name, natural sowie die transformierte Geometrie des Buffers als Spalte geom dar.
- Im Polygondatensatz gibt es unter dem Namen 'Greifensee' nicht nur einen Polygon. Filtern Sie in der Abfrage nur den See.
- Tipp: Zum Filtern des Sees können Sie die Spalte 'natural' verwenden.
-  vgl: https://wiki.openstreetmap.org/wiki/Map_features

<b style="color: gray;">(max. erreichbare Punkte: 8)</b>

In [30]:
# Engine für Datenbankverbindung erstellen


# Ergänzen Sie die SQL-Abfrage, um die Aufgabe zu lösen
sql = """SELECT 
            osm_id,
            "name",
             "natural", ST_TRANSFORM(ST_UNION(ST_Buffer(p.way::geometry, 500)), 4326) AS geom
        FROM public.planet_osm_polygon AS p
        WHERE
            "name" = 'Greifensee' and
            "natural" = 'water'
			GROUP BY osm_id, \"name\", \"natural\""""

# Ergebnis in GeoDataFrame abspeichern
gdf_lake = gpd.GeoDataFrame.from_postgis(sql, engine)

# Datenbankverbindung trennen


# Zeigen des GeoDataFrames
gdf_lake

,osm_id,name,natural,geom
0,-1678346,Greifensee,water,"POLYGON ((8.6478 47.36811, 8.64782 47.3682, 8...."


##### <b>Kartendarstellung Ergebnis (nur anpassen, falls in der Aufgabe danach gefragt wird)</b>

In [31]:
# Projektion definieren (WGS84)
if gdf_lake.crs is None:
    gdf_lake.set_crs(epsg=4326, inplace=True)
else:
    pass

# Latitude und Longitude für die Zentrierung der Karte ermitteln
centroids = gdf_lake.geometry.centroid
lon = centroids.x.mean()
lat = centroids.y.mean()

# Initialisieren der Map
m = folium.Map(location=[lat, lon], 
               zoom_start=13, 
               tiles='ESRIWorldImagery')

# Map settings
folium.Choropleth(
    geo_data=gdf_lake,
    name='map',
    fill_color='greenyellow',
    fill_opacity=0.5,
    line_opacity=0.5
).add_to(m)

folium.LayerControl().add_to(m)

# Save the map to an HTML file
m.save('maps/map_lake.html')
print("Die Karte wurde unter 'maps/map_lake.html' gespeichert.")

Die Karte wurde unter 'maps/map_lake.html' gespeichert.


#### <b>Aufgabe (6): Erstellen Sie eine Abfrage der landwirtschaftlichen Nutzflächen in den Grenzen der Stadt Zürich.</b>
<b>Details zur Aufgabenstellung:</b>
- Sie finden die benötigten Daten in der Tabelle 'planet_osm_polygon'.
- Landwirtschaftliche Nutzflächen sollen in dieser Aufgabe sowohl 'farmland' als auch 'meadows' beinhalten.
- Selektieren Sie die landwirtschaftlichen Nutzflächen über die Gemeindefläche.
- Berechen Sie die Fläche der landwirtschaftlichen Nutzflächen und speichern diese in einer Spalte 'area_ha' ab.
- Stellen Sie in der Ergebnistabelle die Spalten: osm_id, landuse, area_ha sowie die transformierte Geometrie als geom dar.
- Wählen Sie eine Satelliten Karte von ESRI als Hintergrundkarte (maptile).
- Ordnen Sie die Ergebnistabelle nach area_ha absteigend.
- Tipp: Im Unterricht wurde eine ähnliche Abfrage mit Waldflächen (forest) behandelt.
- vgl: https://wiki.openstreetmap.org/wiki/Map_features

<b style="color: gray;">(max. erreichbare Punkte: 8)</b>

In [39]:
# Engine für Datenbankverbindung erstellen


# Ergänzen Sie die SQL-Abfrage, um die Aufgabe zu lösen
sql = """WITH zurich_boundary AS (
    SELECT way
    FROM planet_osm_polygon
    WHERE boundary = 'administrative'
    AND admin_level = '8'
    AND name = 'Zürich'
)
SELECT 
            p.osm_id, p.landuse,
            ST_Area(ST_Transform(p.way, 32632)) / 10000 AS area_ha,
                st_transform(p.way, 4326) AS geom
        FROM public.planet_osm_polygon AS p
        WHERE
            p.landuse IN ('farmland', 'meadows')
			AND ST_Within(ST_Transform(p.way, 4326), (SELECT ST_Transform(way, 4326) FROM zurich_boundary))
		
            ORDER BY area_ha DESC
            """

# Ergebnis in GeoDataFrame abspeichern
gdf_agriculture = gpd.GeoDataFrame.from_postgis(sql, engine)

# Datenbankverbindung trennen


# Zeigen des GeoDataFrames
gdf_agriculture.head(10)

,osm_id,landuse,area_ha,geom
0,1419414136,farmland,14.229945,"POLYGON ((8.51673 47.42273, 8.51791 47.42244, ..."
1,123083907,farmland,13.380850,"POLYGON ((8.5702 47.40217, 8.57021 47.40208, 8..."
2,37201232,farmland,13.357745,"POLYGON ((8.50778 47.40498, 8.51054 47.40417, ..."
3,425913586,farmland,12.912359,"POLYGON ((8.52386 47.41738, 8.52397 47.41716, ..."
4,48565300,farmland,12.572642,"POLYGON ((8.59647 47.36558, 8.59654 47.36535, ..."
5,256312662,farmland,12.355890,"POLYGON ((8.52551 47.42578, 8.52695 47.42573, ..."
6,37208273,farmland,11.556099,"POLYGON ((8.49829 47.40814, 8.4991 47.40778, 8..."
7,258364963,farmland,8.987225,"POLYGON ((8.49773 47.42715, 8.49782 47.4271, 8..."
8,122772656,farmland,7.786949,"POLYGON ((8.57665 47.39778, 8.57719 47.39744, ..."
9,256312665,farmland,7.704376,"POLYGON ((8.51924 47.42997, 8.52053 47.42996, ..."


##### <b>Kartendarstellung Ergebnis (nur anpassen, falls in der Aufgabe danach gefragt wird)</b>

In [40]:
# Projektion definieren (WGS84)
if gdf_agriculture.crs is None:
    gdf_agriculture.set_crs(epsg=4326, inplace=True)
else:
    pass

# Latitude und Longitude für die Zentrierung der Karte ermitteln
centroids = gdf_agriculture.geometry.centroid
lon = centroids.x.mean()
lat = centroids.y.mean()

# Initialisieren der Map
m = folium.Map(location=[lat, lon], 
               zoom_start=13, 
               tiles='EsriWorldImagery')

# Map settings
folium.GeoJson(
    gdf_agriculture,
    name='map',
).add_to(m)

folium.LayerControl().add_to(m)

# Save the map to an HTML file
m.save('maps/map_farmland_meadow.html')
print("Die Karte wurde unter 'maps/map_farmland_meadow.html' gespeichert.")

Die Karte wurde unter 'maps/map_farmland_meadow.html' gespeichert.


#### <b>Aufgabe (7): Erstellen Sie eine Abfrage der Hotels in einem Radius von 5km um den Flughafen Zürich.</b>
<b>Details zur Aufgabenstellung:</b>
- Verwenden Sie die Tabelle 'planet_osm_point'.
- Berechnen Sie in der Abfrage die Distanz jedes Hotels zum Flughafen in Metern als Spalte 'distance_meters'.
- Stellen Sie in der Ergebnistabelle die Spalten: osm_id, tourism, name, distance_meters, sowie die transformierte Geometrie als geom dar.
- Wählen Sie eine Satelliten Karte von ESRI als Hintergrundkarte (maptile).
- Integrieren Sie in die Kartendarstellung den Namen (Spalte 'name') der Hotels als Popup.
- Tipp: Hotels sind mit dem key:value Paar tourism='hotel' in der OpenStreetMap Map-Feature Übersicht angegeben.
- vgl: https://wiki.openstreetmap.org/wiki/Map_features

<b style="color: gray;">(max. erreichbare Punkte: 8)</b>

In [42]:
# Engine für Datenbankverbindung erstellen


# Ergänzen Sie die SQL-Abfrage, um die Aufgabe zu lösen
sql = """SELECT
    p.osm_id,
    p.tourism,
    p.name,
    ST_Distance(
        ST_Transform(p.way, 4326)::geography,
        ST_SetSRID(ST_MakePoint(8.548056,47.458056), 4326)::geography
    ) AS distance_meters,
	ST_TRANSFORM(p.way, 4326) AS geom
FROM
    public.planet_osm_point AS p
WHERE
    p.tourism = 'hotel'
    AND ST_DWithin(
        ST_Transform(p.way, 4326)::geography,
        ST_SetSRID(ST_MakePoint(8.548056, 47.458056), 4326)::geography,
        5000
    )
ORDER BY distance_meters;"""

# Ergebnis in GeoDataFrame abspeichern
gdf_hotels = gpd.GeoDataFrame.from_postgis(sql, engine)

# Datenbankverbindung trennen


# Zeigen des GeoDataFrames
gdf_hotels.head(10)


,osm_id,tourism,name,distance_meters,geom
0,291687394,hotel,Radisson Hotel Zurich Airport,986.742589,POINT (8.54014 47.45099)
1,1060256103,hotel,Holiday Inn Express Zürich Airport,1072.781275,POINT (8.53715 47.45186)
2,8607233055,hotel,Transit Hotel & Dayrooms,1237.185026,POINT (8.56018 47.45056)
3,9553976171,hotel,Capsule Hotel Alpine Garden,1291.028074,POINT (8.5627 47.45204)
4,3086649239,hotel,Dorint Airport-Hotel Zürich,2524.037847,POINT (8.56559 47.43872)
5,3783549518,hotel,STAY@Zurich Airport,2574.228685,POINT (8.56747 47.43901)
6,6286669885,hotel,Welcome Inn,2716.715413,POINT (8.5816 47.44914)
7,12072254954,hotel,Sleepz,2787.377518,POINT (8.58278 47.44947)
8,6286669886,hotel,Fly away,2870.625839,POINT (8.58426 47.45007)
9,4269704591,hotel,Apart-Hotel Zürich by Hilton,2927.703298,POINT (8.57373 47.43831)


In [43]:
# Projektion definieren (WGS84)
if gdf_hotels.crs is None:
    gdf_hotels.set_crs(epsg=4326, inplace=True)
else:
    pass

# Latitude und Longitude für die Zentrierung der Karte ermitteln
centroids = gdf_hotels.geometry.centroid
lon = centroids.x.mean()
lat = centroids.y.mean()

# Initialisieren der Map
m = folium.Map(location=[lat, lon], 
               zoom_start=13, 
               tiles='EsriWorldImagery')

# Map settings
folium.GeoJson(
    gdf_hotels,
    name='map',
    popup=folium.GeoJsonPopup(fields=['name'])
).add_to(m)

folium.LayerControl().add_to(m)

# Save the map to an HTML file
m.save('maps/map_hotels.html')
print("Die Karte wurde unter 'maps/map_hotels.html' gespeichert.")

Die Karte wurde unter 'maps/map_hotels.html' gespeichert.


### Jupyter notebook --footer info-- (please always provide this at the end of each notebook)

In [44]:
import os
import platform
import socket
from platform import python_version
from datetime import datetime

print('-----------------------------------')
print(os.name.upper())
print(platform.system(), '|', platform.release())
print('Datetime:', datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
print('Python Version:', python_version())
print('IP Address:', socket.gethostbyname(socket.gethostname()))
print('-----------------------------------')

-----------------------------------
POSIX
Linux | 6.8.0-1030-azure
Datetime: 2025-10-06 08:08:26
Python Version: 3.12.1
IP Address: 127.0.0.1
-----------------------------------
